<a href="https://colab.research.google.com/github/BorseGaurav95/Random-Data-Generation-tool-in-python/blob/main/Data_generation_script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Developed by Gaurav Borse

In [ ]:
import subprocess
import random as r
import base64
from datetime import datetime
from dateutil import parser
from google.cloud import bigquery
from tqdm import tqdm
from google.oauth2 import service_account
from shapely.geometry import Point
from google.colab import files

def install(package):
    subprocess.check_call([
        "pip", "install", package
    ])

install("faker")

from faker import Faker
fake = Faker()
Faker.seed(99)


json_file_path=""
data = {}
count = 0
col_name=[]


def lat_lng(col_name1):
  col_data = "fake."+col_name1+'()'
  return float(eval(col_data))

def phone_num():
  ph_no=[]
  ph_no.append(r.randint(6, 9))  
  for i in range(1, 10):
      ph_no.append(r.randint(0, 9))
  res = str("".join(map(str, ph_no)))
  return res


def zip_data(col_name1):
  col_data = "fake."+col_name1+'()'
  byte_ch = base64.b64encode(bytes(str(eval(col_data)), 'utf-8')) 
  return byte_ch.decode('utf-8')


def date_time_(col_name1):
  col_data = "fake."+col_name1+'()'
  str_date=str(eval(col_data))
  return str_date

def big_num_():
   return r.uniform(-5.7896044618658097711785492504343953926634992332820282019728792003956564819968E+38,5.7896044618658097711785492504343953926634992332820282019728792003956564819968E+38)
  
def record_data_():
  record = {'name': str(fake.name()), 'age': r.randint(1, 99), 'gender':r.choice(['male','female'])}
  return record

def geography_data():
  return str(Point([float(fake.longitude()), float(fake.latitude())]))


print("-------------------------------------------------------------------")
print("Select GCP Service account Json key file = ")
# Prompt the user to select multiple files
uploaded = files.upload()

key, value = list(uploaded.items())[0]
json_file_path="/content/"+key;

print("-------------------------------------------------------------------")
input_table=input("Enter BigQuery table name (Project_id.Dataset_name.Table_name) = ")
print("-------------------------------------------------------------------")
n_record=int(input("Enter record count = "))
print("-------------------------------------------------------------------")


credentials = service_account.Credentials.from_service_account_file(
json_file_path)

tb_name=input_table.split(".")
project_id = tb_name[0]
dataset_id = tb_name[1]
table_id = tb_name[2]

client = bigquery.Client(credentials= credentials,project=project_id)

query_job = client.query("""
   SELECT column_name 
   FROM """ + project_id + """.""" + dataset_id + """.INFORMATION_SCHEMA.COLUMNS
    where table_name = '""" + table_id + """'
    """)

results = query_job.result()

for row in results:
  col_name.append(row.column_name) 

arr_len=len(col_name)

for i in range(n_record):
    for j in range(arr_len):  
       col_name1 = col_name[j]
  
       if((col_name1=='latitude') or (col_name1=='longitude')):          
          data[col_name1.title()] = lat_lng(col_name1)

       elif((col_name1=='phone_number')):    
          data[col_name1.title()] = phone_num()

       elif((col_name1=='zip')):
          data[col_name1.title()] = zip_data(col_name1)

       elif((col_name1=='date_time')):
          data[col_name1.title()] = date_time_(col_name1)

       elif((col_name1=='latlng')):
          data[col_name1.title()] = geography_data()

       elif((col_name1=='big_num')):
          data[col_name1.title()] = big_num_()
       
       elif((col_name1=='record_data')):
          data[col_name1.title()] = record_data_()

       else:   
          col_data = "fake."+col_name1+'()'
          data[col_name1.title()] = eval(col_data)

       rows_to_insert = [data]
    
       
    errors = client.insert_rows_json(input_table, rows_to_insert)  # Make an API request.
    count=count+1
    
    if errors == []:
      pass
    else:
      print("Encountered errors while inserting rows: {}".format(errors))
      break

for i in tqdm(range(count), desc="Loading..."):
      pass


-------------------------------------------------------------------
Select GCP Service account Json key file = 


Saving wwbq-cartos1-255e5ccf0139.json to wwbq-cartos1-255e5ccf0139 (1).json
-------------------------------------------------------------------
Enter BigQuery table name (Project_id.Dataset_name.Table_name) = wwbq-cartos1.small_performance.data
-------------------------------------------------------------------
Enter record count = 10
-------------------------------------------------------------------


Loading...: 100%|██████████| 10/10 [00:00<00:00, 90394.48it/s]


In [ ]:
from google.cloud import bigquery

project_id="wwbq-cartos1"
cred = service_account.Credentials.from_service_account_file("/content/wwbq-cartos1.json")

client = bigquery.Client(credentials= credentials,project=project_id)
# Create a BigQuery client


# Set the name of the table and the old and new column names
table_name = 'wwbq-cartos1.Test_DG.a'
old_column_name = 'latitude'
new_column_name = 'new_column_name'

# Construct the query
query_job = client.query("""
   SELECT * FROM ALTER TABLE """+table_name+""" CHANGE COLUMN """+old_column_name+""" """+new_column_name+""" float """)


# Execute the query
query_job = client.query(query_job)

# Wait for the query to complete
query_job.result()


In [ ]:
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build

# Set the project ID and table ID
project_id = 'wwbq-cartos1'
dataset_id = 'Test_DG'
table_id = 'a'

# Set the new column name and data type
new_column_name = 'new_column_name'
new_column_type = 'float'

# Create a service client
cred = service_account.Credentials.from_service_account_file("/content/wwbq-cartos1.json")
service = build('bigquery', 'v2', credentials=cred)

# Get the table metadata
table = service.tables().get(projectId=project_id, datasetId=dataset_id, tableId=table_id).execute()

# Get the list of existing columns
columns = table['schema']['fields']

# Find the index of the column you want to rename
column_index = -1
for i, column in enumerate(columns):
  if column['name'] == 'latit':
    column_index = i
    
    break


# If the column was found, update its name and data type
if column_index >= 0:
  columns[column_index]['name'] = new_column_name
  print("yes")
  columns[column_index]['type'] = new_column_type

# Update the table schema
table['schema']['fields'] = columns
service.tables().patch(projectId=project_id, datasetId=dataset_id, tableId=table_id, body=table).execute()


{'kind': 'bigquery#table',
 'etag': '5F752ofsd5zRZOxIpg1Tuw==',
 'id': 'wwbq-cartos1:Test_DG.a',
 'selfLink': 'https://bigquery.googleapis.com/bigquery/v2/projects/wwbq-cartos1/datasets/Test_DG/tables/a',
 'tableReference': {'projectId': 'wwbq-cartos1',
  'datasetId': 'Test_DG',
  'tableId': 'a'},
 'schema': {'fields': [{'name': 'latitude',
    'type': 'FLOAT',
    'mode': 'NULLABLE'}]},
 'numBytes': '0',
 'numLongTermBytes': '0',
 'numRows': '0',
 'creationTime': '1672507471746',
 'lastModifiedTime': '1672507471798',
 'type': 'TABLE',
 'location': 'US',
 'numTimeTravelPhysicalBytes': '1248',
 'numTotalLogicalBytes': '0',
 'numActiveLogicalBytes': '0',
 'numLongTermLogicalBytes': '0',
 'numTotalPhysicalBytes': '1248',
 'numActivePhysicalBytes': '1248',
 'numLongTermPhysicalBytes': '0'}

In [ ]:
import subprocess
import random as r
import base64
from datetime import datetime
from dateutil import parser
from google.cloud import bigquery
from tqdm import tqdm
from google.oauth2 import service_account
from shapely.geometry import Point
from google.colab import files

def install(package):
    subprocess.check_call([
        "pip", "install", package
    ])

install("faker")

from faker import Faker

json_file_path=""
print("-------------------------------------------------------------------")
print("Select GCP Service account Json key file = ")
# Prompt the user to select multiple files
uploaded = files.upload()

key, value = list(uploaded.items())[0]
json_file_path="/content/"+key;

 
print("-------------------------------------------------------------------")
input_table=input("Enter BigQuery table name (Project_id.Dataset_name.Table_name) = ")
print("-------------------------------------------------------------------")
n_record=int(input("Enter record count = "))
print("-------------------------------------------------------------------")


credentials = service_account.Credentials.from_service_account_file(
json_file_path)

tb_name=input_table.split(".")
project_id = tb_name[0]
dataset_id = tb_name[1]
table_id = tb_name[2]

client = bigquery.Client(credentials= credentials,project=project_id)

query_job = client.query("""
   SELECT column_name 
   FROM """ + project_id + """.""" + dataset_id + """.INFORMATION_SCHEMA.COLUMNS
    where table_name = '""" + table_id + """'
    """)


results = query_job.result()

col_name=[]

for row in results:
  col_name.append(row.column_name) 

arr_len=len(col_name)

data = {}
count = 0

fake = Faker()
Faker.seed(99)

for i in range(n_record):
    for j in range(arr_len):  
       col_name1 = col_name[j]
  
       if((col_name1=='latitude') or (col_name1=='longitude')):
          col_data = "fake."+col_name1+'()'
          data[col_name1.title()] = float(eval(col_data))
       elif((col_name1=='phone_number')):
          # col_data = "fake."+col_name1+'()'
          # data[col_name1.title()] = eval(col_data)
          ph_no=[]
          ph_no.append(r.randint(6, 9))  
          for i in range(1, 10):
              ph_no.append(r.randint(0, 9))
          res = str("".join(map(str, ph_no)))
          data[col_name1.title()] = res
       elif((col_name1=='zip')):
          col_data = "fake."+col_name1+'()'
          byte_ch = base64.b64encode(bytes(str(eval(col_data)), 'utf-8'))
          data[col_name1.title()] = byte_ch.decode('utf-8')
       elif((col_name1=='date_time')):
          col_data = "fake."+col_name1+'()'
          str_date=str(eval(col_data))
          data[col_name1.title()] = str_date
          #data[col_name1.title()] = parser.parse(str_date)
          #print(data[col_name1.title()])
          #data[col_name1.title()] =  datetime.strptime(str_date, '%')
       elif((col_name1=='pydecimal')):
          col_data = "fake."+col_name1+'()'
          big_num=str(eval(col_data))
          data[col_name1.title()] = big_num
       elif((col_name1=='latlng')):
          data[col_name1.title()] = str(Point(
          [float(fake.longitude()), float(fake.latitude())]))
       elif((col_name1=='big_num')):
          data[col_name1.title()] = r.uniform(-5.7896044618658097711785492504343953926634992332820282019728792003956564819968E+38,5.7896044618658097711785492504343953926634992332820282019728792003956564819968E+38)
       elif((col_name1=='record_data')):
          data[col_name1.title()] = record = {'name': str(fake.name()), 'age': r.randint(1, 99), 'gender':r.choice(['male','female'])}

       else:   
          col_data = "fake."+col_name1+'()'
          data[col_name1.title()] = eval(col_data)
       rows_to_insert = [data]
    # print(rows_to_insert)
       
    errors = client.insert_rows_json(input_table, rows_to_insert)  # Make an API request.
    count=count+1
    if errors == []:
      pass
    else:
      print("Encountered errors while inserting rows: {}".format(errors))
      break
for i in tqdm(range(count), desc="Loading..."):
      pass


-------------------------------------------------------------------
Select GCP Service account Json key file = 


Saving wwbq-cartos1.json to wwbq-cartos1.json
-------------------------------------------------------------------
Enter BigQuery table name (Project_id.Dataset_name.Table_name) = wwbq-cartos1.Test_DG.a
-------------------------------------------------------------------
Enter record count = 1
-------------------------------------------------------------------


Loading...: 100%|██████████| 1/1 [00:00<00:00, 6932.73it/s]
